## loading secret API keys from your environment so your program can use them

#### Getting started With Langchain And Open AI

In this quickstart we'll see how to:

- Get setup with LangChain, LangSmith and LangServe
- Use the most basic and common components of LangChain: prompt templates, models, and output parsers.
- Build a simple application with LangChain
- Trace your application with LangSmith
- Serve your application with LangServe

In [1]:
# Import the 'os' module to interact with the operating system
import os 

# Import the 'load_dotenv' function from the 'dotenv' library
from dotenv import load_dotenv

# Load environment variables from a .env file into the program
load_dotenv()

# Set the OpenAI API key from the environment variable
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Set the LangChain API key from the environment variable
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# Enable LangChain Tracing v2 for tracking
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Set the project name for LangChain from the environment variable
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")


In [2]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")

print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000014F35FD5BD0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000014F35FF4970> root_client=<openai.OpenAI object at 0x0000014F3352B5E0> root_async_client=<openai.AsyncOpenAI object at 0x0000014F35FD5C90> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [3]:
result=llm.invoke("What is the capital of France?")
print(result)

content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9bddfca6e2', 'id': 'chatcmpl-BYPWEf0Yh1kp6m0jQ9GVAQH9b9LvE', 'finish_reason': 'stop', 'logprobs': None} id='run-3a086111-2bc3-4dc7-8b5c-0cddc951e48d-0' usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [4]:
result.content

'The capital of France is Paris.'

# Chat Prompt Template in Langchain
## Definition
A Chat Prompt Template is a ready-made message format with blanks (called placeholders) that you can fill with different information before sending it to an AI chatbot.

## Uses
Reuse messages with different inputs without rewriting the whole message.

Keep conversations consistent and organized.

Make your code cleaner and easier to maintain.

Fill in user data dynamically (like name, question, or topic) to create personalized AI messages.

## Example
Template:
"Hello, my name is {name}. Can you help me with {topic}?"

## Fill with data:
name = "Alice"
topic = "Python programming"

## Final message:
"Hello, my name is Alice. Can you help me with Python programming?"

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt= ChatPromptTemplate.from_messages(
    [
        ("system","you are an expert Ai Engineer.Provide me answers based on the questions"),
        ("user","{input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are an expert Ai Engineer.Provide me answers based on the questions'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

# Using this prompt template along with the LLM model by creating chain

# Chain in Langchain
## Definition
A Chain connects multiple steps in order. The output of one step goes into the next step automatically.

## Uses
To do many things one after another.

To automate tasks with the AI.

To keep your code clean and organized.

## Example
If you want to:

Ask AI to summarize something.

Then translate that summary.

A Chain helps you do both steps easily, one after the other.

In [ ]:
chain=prompt | llm

response=chain.invoke({"input":"who is PM of India?"})
print(response)

content='As of my last update in October 2023, the Prime Minister of India is Narendra Modi. However, please verify with a current source as this information may have changed after my last update.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 31, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BYPykS4s1SsYpFXuIbr8InANbUktm', 'finish_reason': 'stop', 'logprobs': None} id='run-185c7ff9-6f5d-47c8-a0fe-6448bb6d03df-0' usage_metadata={'input_tokens': 31, 'output_tokens': 39, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [8]:
response.content

'As of my last update in October 2023, the Prime Minister of India is Narendra Modi. However, please verify with a current source as this information may have changed after my last update.'

In [9]:
type(response)

langchain_core.messages.ai.AIMessage

# Output Parsers
## Definition
An Output Parser takes the raw text response from an AI model and cleans it up or changes its format so your program can use it easily.

## Why use Output Parsers?
AI sometimes gives extra text or formatting you don’t want.

Parsers help you get only the useful part, like JSON, numbers, or simple strings.

Makes it easier to work with AI answers in your code.

#StrOutputParser
## What is it?
The StrOutputParser simply returns the AI response as a clean string without any extra processing.

## When to use StrOutputParser?
When you just want the raw text reply from the AI.

When you don’t need to extract or change anything from the output.

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()

chain=prompt | llm | output_parser
response=chain.invoke({"input":"who is PM of India?"})
print(response)

As of October 2023, the Prime Minister of India is Narendra Modi.


In [11]:
# now you can observe that ,there are no unnessasary keys in the response,you got only the content(raw text)
# that is the power of output parsers